In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!bash drive/My\ Drive/datawithcred.sh

100% 2.29G/2.29G [00:28<00:00, 131MB/s]
100% 2.29G/2.29G [00:28<00:00, 87.6MB/s]
Cloning into 'MSc-Project'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 413 (delta 56), reused 90 (delta 26), pack-reused 292
Receiving objects: 100% (413/413), 8.65 MiB | 33.92 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import glob
import pickle
from pathlib import Path
import gc
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from collections import Counter

In [ ]:
data_dir = Path("/content/chest_xray")

train_dir = data_dir / "train"

test_dir = data_dir / "test"

val_dir = data_dir / "val"

train_dir_normal = train_dir / "NORMAL"
train_dir_pne = train_dir / "PNEUMONIA"

test_dir_normal = test_dir / "NORMAL"
test_dir_pne = test_dir / "PNEUMONIA"

normal_val = val_dir / "NORMAL"
val_dir_pne = val_dir / "PNEUMONIA"

In [ ]:
IMAGE_SIZE = 224

In [ ]:
def img_to_array(path_for_class_one, path_for_class_zero):
  X = []
  y = []
  
  for image in list(path_for_class_one.glob("*.jpeg")):
    img = keras.preprocessing.image.load_img(image, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img = keras.preprocessing.image.img_to_array(img)
    img = np.array(img).ravel()
    img = img /255
    X.append(img)
    y.append(1)
    
    
  for image in list(path_for_class_zero.glob("*.jpeg")):
    img = keras.preprocessing.image.load_img(image, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img = keras.preprocessing.image.img_to_array(img)
    img = np.array(img).ravel()
    img = img /255
    X.append(img)
    y.append(0)
    
  X = np.array(X)
  y = np.array(y)
    
  return X, y

In [ ]:
X, y = img_to_array(train_dir_pne, train_dir_normal)

In [ ]:
#Randomly shuffle the dataset
random_index = np.random.permutation(X.shape[0])
X = X[random_index]
y = y[random_index]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
del X
del y
gc.collect()

21

In [ ]:
del X_test
del y_test
gc.collect()

0

In [ ]:
parameters = {'n_estimators':[100, 500], 'max_features':['sqrt', 'log2']}
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters, cv=3, verbose=3)
clf.fit(X, y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] max_features=sqrt, n_estimators=100 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . max_features=sqrt, n_estimators=100, score=0.953, total= 1.6min
[CV] max_features=sqrt, n_estimators=100 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


[CV] . max_features=sqrt, n_estimators=100, score=0.955, total= 1.8min
[CV] max_features=sqrt, n_estimators=100 .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.4min remaining:    0.0s


[CV] . max_features=sqrt, n_estimators=100, score=0.951, total= 1.8min
[CV] max_features=sqrt, n_estimators=500 .............................
[CV] . max_features=sqrt, n_estimators=500, score=0.955, total= 8.1min
[CV] max_features=sqrt, n_estimators=500 .............................
[CV] . max_features=sqrt, n_estimators=500, score=0.959, total= 8.9min
[CV] max_features=sqrt, n_estimators=500 .............................
[CV] . max_features=sqrt, n_estimators=500, score=0.957, total= 8.7min
[CV] max_features=log2, n_estimators=100 .............................
[CV] . max_features=log2, n_estimators=100, score=0.951, total=   7.0s
[CV] max_features=log2, n_estimators=100 .............................
[CV] . max_features=log2, n_estimators=100, score=0.950, total=   5.9s
[CV] max_features=log2, n_estimators=100 .............................
[CV] . max_features=log2, n_estimators=100, score=0.955, total=   6.0s
[CV] max_features=log2, n_estimators=500 .............................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 32.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
clf.best_params_

{'max_features': 'sqrt', 'n_estimators': 500}

In [ ]:
clf.best_score_

0.9568634937608813

In [ ]:
with open("drive/My Drive/models/rf_sqrt_500.pickle", "wb") as model:
    pickle.dump(clf, model)

In [ ]:
Counter(y)

Counter({0: 1341, 1: 3875})

In [ ]:
2624 / (2624 + 870)

0.7510017172295363